1. WEBSCRAPING REVIEWS FROM JUMIA

link to page:https://www.reviews.io/company-reviews/store/jumia

To web scrape JUMIA product reviews I will be using using Python with selenium libraries. Below is a step-by-step guide and code snippet that extracts the reviews, ratings, and other details from the HTML.

In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

In [26]:
#Selenium configuration
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_service = Service('.\chromedriver.exe')

driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

<>:4: SyntaxWarning: invalid escape sequence '\c'
<>:4: SyntaxWarning: invalid escape sequence '\c'
C:\Users\nolimitmide\AppData\Local\Temp\ipykernel_7780\3728237949.py:4: SyntaxWarning: invalid escape sequence '\c'
  chrome_service = Service('.\chromedriver.exe')


In [27]:
#scraping reviews from the page
def scrape_reviews_from_page(url):
    driver.get(url)
    time.sleep(5) #loading the page, so wait

    reviews = []

    review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.Review')
    for review in review_elements:
        try:
            author = review.find_element(By.CSS_SELECTOR, 'a.Review__author').text
            rating = review.find_element(By.CSS_SELECTOR, 'div.Rating__stars').get_attribute('innerHTML').count('stars__icon--100')  # Count filled stars
            review_text = review.find_element(By.CSS_SELECTOR, 'span.Review__body').text
            date = review.find_element(By.CSS_SELECTOR, 'div.Review__dateSource').text
            
            reviews.append({
                'Rating': rating,
                'Review': review_text,
                'Date': date,
                'Author': author
            })
        except Exception as e:
            print(f"Error extracting review data: {e}")

    return reviews

In [28]:
#Defining the function to scrape multiple pages so we can get 600 reviews
def scrape_reviews(url, max_reviews=600):
    all_reviews = []
    page_number = 1

    while len(all_reviews) < max_reviews:
        page_url = f"{url}?page={page_number}"
        reviews = scrape_reviews_from_page(page_url)

        if not reviews:
            break
        
        all_reviews.extend(reviews)
        page_number += 1

        #sleepto avoid server overload
        time.sleep(5)

    return all_reviews[:max_reviews]

In [29]:
#URL
base_url = 'https://www.reviews.io/company-reviews/store/jumia'
reviews = scrape_reviews(base_url)

In [31]:
#Converting to excel format
df = pd.DataFrame(reviews)
df.to_csv('reviews.csv', index=False)
print("Reviews have been saved to reviews.csv")

Reviews have been saved to reviews.csv


In [32]:
#stop the browser
driver.quit()